In [6]:
!yes "" | pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, http://nexus3/repository/pypi-private/simple
User for nexus3: WARNING: 401 Error, Credentials not correct for http://nexus3/repository/pypi-private/simple/hydra-core/
DEPRECATION: pytorch-lightning 1.8.1 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
yes: standard output: Broken pipe


In [30]:
from src.model.sam import CustomSamAutomaticMaskGenerator, load_sam
from segment_anything.modeling.sam import Sam
model_type = "vit_h"
checkpoint_dir =  "datasets/bop23_challenge/pretrained/segment-anything"
sam_model = load_sam(model_type, checkpoint_dir)
custom_sam_model = CustomSamAutomaticMaskGenerator(sam=sam_model)


In [31]:
from src.model.dinov2 import CustomDINOv2
import torch
descriptor_model = CustomDINOv2(
        model_name="dinov2_vitl14",
        model=torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14'),
        token_name="x_norm_clstoken",
        image_size=224,
        chunk_size=16,
        descriptor_width_size=640,
        patch_size=14,
)

Using cache found in /home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main


In [49]:
from src.model.detector import CNOS
from src.model.loss import PairwiseSimilarity
cnos_model = CNOS(
    segmentor_model = custom_sam_model,
    descriptor_model = descriptor_model,
    onboarding_config = {
        "rendering_type": "pbr", # or pyrender (or onboarding_static or onboarding_dynamic for model-free)
        "reset_descriptors": False,
        "level_templates": 0, # 0 is coarse, 1 is medium, 2 is dense
    },
    matching_config = {
        "metric" : PairwiseSimilarity(metric="cosine", chunk_size=16),
        "aggregation_function": "avg_5",
        "confidence_thresh": 0.01
    },
    post_processing_config = {
        "mask_post_processing" : {
            "min_box_size": 0.05, # relative to image size 
            "min_mask_size": 3e-4, # relative to image size
        },
        "nms_thresh": 0.25
    },
    log_interval = 5,
    log_dir = "datasets/bop23_challenge/results/cnos_exps/test_cnos"

)

In [ ]:
def move_to_device(self):
    self.descriptor_model.model = self.descriptor_model.model.to(self.device)
    self.descriptor_model.model.device = self.device
    # if there is predictor in the model, move it to device
    if hasattr(self.segmentor_model, "predictor"):
        self.segmentor_model.predictor.model = (
            self.segmentor_model.predictor.model.to(self.device)
        )
    else:
        self.segmentor_model.model.setup_model(device=self.device, verbose=True)
    logging.info(f"Moving models to {self.device} done!")

In [50]:
def move_to_device(segmentor_model, device="cuda:0"):
    # if there is predictor in the model, move it to device
    if hasattr(segmentor_model, "predictor"):
        segmentor_model.predictor.model = (
            segmentor_model.predictor.model.to(device)
        )
    else:
        segmentor_model.model.setup_model(device=device, verbose=True)
move_to_device(custom_sam_model)

In [53]:
cnos_model.device

device(type='cpu')

In [44]:
from PIL import Image
import torchvision.transforms as T
import torch
import numpy as np

# rgb_path = "datasets/bop23_challenge/results/cnos_exps/visualization/sam_pbr_tless/tless000001/000488.png"
rgb_path = "datasets/bop23_challenge/datasets/icbin/test/000001/rgb/000001.png"
dataset_name = rgb_path.split('/')[-5]
image = Image.open(rgb_path)

inv_rgb_transform = T.Compose(
    [
        T.PILToTensor(),
        T.ConvertImageDtype(torch.float32),  # Convert uint8 to float
        T.Normalize(
            mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
            std=[1 / 0.229, 1 / 0.224, 1 / 0.225],
        ),
    ]
)

image_np = (
    inv_rgb_transform(image) # batch["image"][0] 0 to retrun image 3,H,W cos the batch has size of batch_size,3,W,H - where batch_size = 1
    .cpu()
    .numpy()
    .transpose(1, 2, 0)
)
image_np = np.uint8(image_np.clip(0, 1) * 255) # just get image in numpy in range of 0,255

In [48]:
import torch
import torchvision.transforms as T
from tqdm import tqdm
import numpy as np
from PIL import Image
import logging
import os
import os.path as osp
from torchvision.utils import make_grid, save_image
import pytorch_lightning as pl
from src.utils.inout import save_json, load_json, save_json_bop23
from src.model.utils import BatchedData, Detections, convert_npz_to_json
from hydra.utils import instantiate
import time
import glob
from functools import partial
import multiprocessing

# run propoals
proposal_stage_start_time = time.time()
proposals = cnos_model.segmentor_model.generate_masks(image_np)

# init detections with masks and boxes
detections = Detections(proposals)
detections.remove_very_small_detections(
    config=cnos_model.post_processing_config.mask_post_processing
)
# compute descriptors
query_decriptors = cnos_model.descriptor_model(image_np, detections) # descriptor_model = dinov2 # so here is for getting proposal_descriptors
proposal_stage_end_time = time.time()

# matching descriptors
matching_stage_start_time = time.time()
(
    idx_selected_proposals,
    pred_idx_objects,
    pred_scores,
) = cnos_model.find_matched_proposals(query_decriptors)

# update detections
detections.filter(idx_selected_proposals)
detections.add_attribute("scores", pred_scores)
detections.add_attribute("object_ids", pred_idx_objects)
detections.apply_nms_per_object_id(
    nms_thresh=cnos_model.post_processing_config.nms_thresh
)
matching_stage_end_time = time.time()

runtime = (
    proposal_stage_end_time
    - proposal_stage_start_time
    + matching_stage_end_time
    - matching_stage_start_time
)
detections.to_numpy()


# scene_id = batch["scene_id"][0]
# frame_id = batch["frame_id"][0]
# file_path = osp.join(
#     cnos_model.log_dir,
#     f"predictions/{dataset_name}/{cnos_model.name_prediction_file}/scene{scene_id}_frame{frame_id}",
# )

# # save detections to file
# results = detections.save_to_file(
#     scene_id=int(scene_id),
#     frame_id=int(frame_id),
#     runtime=runtime,
#     file_path=file_path,
#     dataset_name=dataset_name,
#     return_results=True,
# )
# # save runtime to file
# np.savez(
#     file_path + "_runtime",
#     proposal_stage=proposal_stage_end_time - proposal_stage_start_time,
#     matching_stage=matching_stage_end_time - matching_stage_start_time,
# )

KeyboardInterrupt: 

Looking in indexes: https://pypi.org/simple, http://nexus3/repository/pypi-private/simple
User for nexus3: WARNING: 401 Error, Credentials not correct for http://nexus3/repository/pypi-private/simple/hydra-core/
DEPRECATION: pytorch-lightning 1.8.1 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
User for nexus3: WARNING: 401 Error, Credentials not correct for http://nexus3/repository/pypi-private/simple/pip/
yes: standard output: Broken pipe
